In [1]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
from matplotlib import rc
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('font',**{'family':'sans-serif',
             'sans-serif':['Helvetica', 'HelveticaNeue', 'Helvetica', 'Arial', 'Droid Sans Mono'],
             'style':'normal',
             'size':8})
rc('xtick', labelsize=8)
rc('ytick', labelsize=8)
rc('text', usetex=True) # Must be True to use LaTeX backend
rc("text.latex", preamble="\\usepackage{helvet}\\usepackage{amsmath}")

import string

from datetime import datetime
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from operator import sub

import os


from tkinter import Tk
from tkinter.filedialog import askopenfilename, askdirectory

In [2]:
def get_aspect(ax):
    # Total figure size
    fig_width, fig_h = ax.get_figure().get_size_inches()
    
    # Axis size on figure
    _, _, w, h = ax.get_position().bounds
    
    # Ratio of display units
    display_ratio = (fig_h * h) / (fig_width * w)
    
    # Ratio of data units
    # Negative over negative because of the order of subtraction
    data_ratio = sub(*ax.get_ylim()) / sub(*ax.get_xlim())

    return np.abs(display_ratio / data_ratio)

In [3]:
def int_to_roman(input):
    """ Convert an integer to a Roman numeral. """
    ints = (1000, 900,  500, 400, 100,  90, 50,  40, 10,  9,   5,  4,   1)
    nums = ('m',  'cm', 'd', 'cd','c', 'xc','l','xl','x','ix','v','iv','i')
    result = []
    for i in range(len(ints)):
        count = int(input / ints[i])
        result.append(nums[i] * count)
        input -= ints[i] * count
    return ''.join(result)

In [4]:
def create_bar(i, vertical_position, work_package):
    
    work_package_start.append(df.Start[i])
    work_package_end.append(df.End[i])
    work_package_duration.append(df.End[i] - df.Start[i])
    work_package_midpoint.append(df.Start[i] + ((df.End[i] - df.Start[i]) / 2))
    bar_vertical_positions.append(vertical_position)
    
    if df.Annotation.isna()[i]: # If annotation is empty
        work_package_annotations.append(int_to_roman(work_package))
    else: # an annotation has been specified
        work_package_annotations.append(int_to_roman(work_package) + r': ' + df.Annotation[i])
        
    if df.Color[i] in ColorScheme:
        bar_color.append(ColorScheme[str(df.Color[i])])
    else: # use grey as color
        bar_color.append('#7f7f7f')
    
    return

In [5]:
# withdraw tkinter root window
root = Tk()
root.attributes("-topmost", True) # raise to be in front of other windows
root.withdraw()

# open dialog box and return the path to the selected file
spreadsheet_path = askopenfilename(initialdir = os.getcwd(),
                                     title = "Select spreadsheet containing Gantt data",
                                     filetypes = (("Spreadsheets","*.xlsx"),("all files","*.*"))) 

print('spreadsheet_path:')
print(spreadsheet_path)

# Read EXCEL spreadsheet
df = pd.read_excel(spreadsheet_path)

spreadsheet_path:
C:/Users/Alaric Taylor-Roffey/Downloads/CRUK_Gantt2.xlsx


In [6]:
plt.close('all')

fig, ax  = plt.subplots(figsize=(11.7,8.27))    # A4 is 11.7 x 8.27 inches
                                                #      (29.7 x 21.0 cm)
                                                # A3 is 16.5 x 11.7 inches
                                                #      (42.0 x 29.7 cm)

# Setting margins for printing and space for labels on left edge
plt.subplots_adjust(left=0.7/29.7, # fraction of total figsize (A4 is 29.7 x 21 cm)
                    right= 20.7/29.7,
                    top=20.3/21,
                    bottom=1.4/21)

# Alphabet list for labelling work packages
alphabet_list = string.ascii_uppercase

# Dictionary of colours referenced by EXCEL spreadsheet
ColorScheme = {'A': '#957DAD',
               'B': '#E0BBE4',
               'C': '#D291BC',
               'D': '#FEC8D8',
               'E': '#FFDFD3',
               'F': '#90C978',
               'G': '#83C6DD',
               'H': '#AFD5AA',
               'I': '#5DB1D1'}

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 842.4x595.44 with 1 Axes>

In [7]:
########################
# HIERACHY (terminology)
#   
# Group
#   Activity
#       Work package
#

# Local variables used for annotations
group = -1 # Parent grouping (only groups 0+ will be converted to text)
activity = -1 # Enumerated line within each group (only activities 0+ will be converted to text)
activity_alpha = '' # e.g. 'a' (arbritraty until set, below)
activity_reference = '' # # e.g. '2b' (arbritraty until set, below)
work_package = int # Enumerated within each line e.g. 2 (arbritraty until set, below)
work_package_roman = '' # e.g. vi (arbritraty until set, below)
vertical_position = -1 # Vertical position, first line will start at +1 as incrementing

# Lists used for plotting
row_title_positions = []
row_titles = []
group_break_positions = []

work_package_start = []
work_package_end = []
bar_vertical_positions = []
bar_color = []
work_package_duration = []
work_package_midpoint = []
work_package_annotations = []

The minimum requirement for the spreadsheet is that it has the following column (titled on row 1):

- `Start`...of type datetime
- `End`...of type datetime
- `LineTitle`...of type string

When `Start`, `End` and `LineTitle` are specified, then a new activity (line) is created... however, if:

- one of either `Start` or `End` is not specified, then the `LineTitle` is assumed to be a Group Title
- `LineTitle` is not specified (but both `Start` or `End` are specified), then the work package is considered to be part of the same activity and therefore sits on the same line.

Additional information can be included in columns titled:

- `Color`...of type string, used to look-up hex color defined in `ColorScheme`
- `Annotation`...of type string
- `ResourceAllocation`

In [8]:
for i in range(df.shape[0]): # Scan through all rows of the spreadsheet
    
    # For debugging output:
    vis_row = i+2
    num_vis_rows = df.shape[0]+1
    #print('Row:', vis_row, 'of', num_vis_rows,'\t')
    
    ################
    # Sense checking
    
    if not df.Start.isna()[i]: # If start date is specified,
        if df.End.isna()[i]: # but end date is empty,
            print("ERROR")
            print("Line", i+2,"specifies an start date but has no end date")
            raise
    
    if df.Start.isna()[i]: # If start date is empty,
        if not df.End.isna()[i]: # but end date is specified,
            print("ERROR!")
            print("Line", i+2,"specifies an end date but has no start date")
            raise
    
    if df.Start.isna()[i]: # If start date is empty,
        if df.End.isna()[i]: # end date is empty
            if df.LineTitle.isna()[i]: # and no line title is specified,
                print("WARNING")
                print("Line", i+2,"is empty and will not be plotted")
    
    if not df.Start.isna()[i]: # If start date is specified,
        if not df.End.isna()[i]: # end date is specified
            if mdates.date2num(df.End[i])-mdates.date2num(df.Start[i]) < 0:
                print("ERROR")
                print("Line", i+2,": task starts after end date...?!")
                raise
     
    ##############
    # Line parsing           
    
    if df.Start.isna()[i]: # If start date is empty,
        if df.End.isna()[i]: # and end date is empty,
            if not df.LineTitle.isna()[i]: # but a title is specified,
                group += 1 # ...so increment the group number
                vertical_position += 1 # and increment the vertical position (new row/line)
                activity = -1 # Reset the activity counter
                
                row_titles.append(df.LineTitle[i]) # LineTitle will be printed unchanged
                row_title_positions.append(vertical_position) # Note the position
                
                group_break_positions.append(vertical_position)
                
                # Degubbing output:
                #print('') # Blank line (divider)
                print('Row:', vis_row, 'of', num_vis_rows,'\t',
                      df.LineTitle[i], '...(group <', group,'>, position <',vertical_position,'>)')
                #print('New group (', group,'): ', df.LineTitle[i]) # ...then the line is a new group title
    
    if not df.Start.isna()[i]: # If start date is specified,
        if not df.End.isna()[i]: # and end date is specified,
            if not df.LineTitle.isna()[i]: # and a title is specified,
                activity += 1 # So, increment the activity counter
                activity_reference = str(group) + alphabet_list[activity]
                vertical_position += 1 # and increment the vertical position (new row/line)
                work_package = 1 # and reset work package counter
                
                if group < 0: # If we have not yet specified a group
                    row_titles.append(df.LineTitle[i]) # LineTitle will be printed unchanged
                else:
                    row_titles.append(r' \textbf{' + activity_reference + ')} ' + df.LineTitle[i].strip()) # add numeric/alpha/roman reference and remove leading and tailing spaces
                row_title_positions.append(vertical_position) # note the position
                
                create_bar(i, vertical_position, work_package)
                
                #Debugging output:
                print('Row:', vis_row, 'of', num_vis_rows,'\t',
                      '\t',df.LineTitle[i],'\t... (activity/position <',vertical_position,'>, work-package <', work_package,'>)') # Then this is a new activity
                
    if not df.Start.isna()[i]: # If start date is specified,
        if not df.End.isna()[i]: # and end date is specified,
            if df.LineTitle.isna()[i]: # but no title is specified,
                
                work_package += 1 # increment the work package number
                
                create_bar(i, vertical_position, work_package)
                
                #Debugging output:
                print('Row:', vis_row, 'of', num_vis_rows,'\t',
                      '\t\t... (same activity, work-package <', work_package,'>)') # Then this is a new activity
                #print("New work package (same activity)") # Then this is a new work package in the same activity (sub-group)

Row: 2 of 63 	 \textbf{WP1: Development of point-of-care biosample prep unit} ...(group < 0 >, position < 0 >)
Row: 3 of 63 	 	 Hardware development for ward use 	... (activity/position < 1 >, work-package < 1 >)
Row: 4 of 63 	 	 \textit{Deliverable 1.1*} 	... (activity/position < 2 >, work-package < 1 >)
Row: 5 of 63 	 	 Software interface and hardware optimisation via end-user human factors feedback. 	... (activity/position < 3 >, work-package < 1 >)
Row: 6 of 63 	 	 \textit{Deliverable 1.2} 	... (activity/position < 4 >, work-package < 1 >)
Row: 7 of 63 	 	 Securitisation and integration of clinical data management  	... (activity/position < 5 >, work-package < 1 >)
Row: 8 of 63 	 	 \textit{Deliverable 1.3} 	... (activity/position < 6 >, work-package < 1 >)
Row: 9 of 63 	 \textbf{WP2: Clinical study setup \& device training} ...(group < 1 >, position < 7 >)
Row: 10 of 63 	 	 Ethics, research approval \& data governance  	... (activity/position < 8 >, work-package < 1 >)
Row: 11 of 6

In [9]:
#Debugging read:

if True:
    #print('row_title_positions',row_title_positions)
    #print('row_titles',row_titles)
    #print('group_break_positions',group_break_positions)
    #print('work_package_start',work_package_start)
    #print('work_package_end',work_package_end)
    #print('bar_vertical_positions',bar_vertical_positions)
    print('bar_color',bar_color)
    #print('work_package_duration',work_package_start)
    #print('work_package_midpoint',work_package_midpoint)
    #print('work_package_annotations',work_package_annotations)
    
if False:
    for i in work_package_start:
        print(mdates.date2num(i))

bar_color ['#957DAD', '#AFD5AA', '#957DAD', '#AFD5AA', '#957DAD', '#AFD5AA', '#E0BBE4', '#AFD5AA', '#E0BBE4', '#AFD5AA', '#E0BBE4', '#AFD5AA', '#E0BBE4', '#AFD5AA', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#D291BC', '#AFD5AA', '#AFD5AA', '#FEC8D8', '#AFD5AA', '#FFDFD3', '#AFD5AA', '#90C978', '#90C978', '#90C978', '#AFD5AA', '#AFD5AA']


In [10]:
###############################
# Plotting resource allocations
    
if False:
    
    all_resources = []
    for i in range(df.shape[0]):
        if not df.ResourceAllocation.isna()[i]: # If resources are specified
            all_resources.extend(df.ResourceAllocation[i].split(";")) # Add each individual resource as seperate item
    
    print('Unfiltered resources:', all_resources)
    
    for i in range(len(all_resources)):
        #print(all_resources[i])
        all_resources[i] = all_resources[i].strip() # Remove leading and trailing spaces
    
    unique_resources = list(set(all_resources))
    #unique_resources.remove('') # Remove blank items
    unique_resources.sort() # Alphabetical sort
    unique_resources.reverse() # Reverse order (as plotting in this order, from Z up to A)
    
    print('Filtered resources:', unique_resources)
    
    resource_position = -1 # Initiate counting from -1 (as )
    for resource in unique_resources:
        print('')
        print('Collating resource: ', resource)
        for i in range(df.shape[0]): # Iterate through all lines of spreadsheet
            if type(df.ResourceAllocation[i]) == str:
                if resource in df.ResourceAllocation[i].split(";"):
                    print(resource, "found on line", i+2, "of", df.shape[0]+1)
                    if not df.Start.isna()[i]: # If start date is specified,
                        if not df.End.isna()[i]: # and end date is specified,
                            if mdates.date2num(df.End[i])-mdates.date2num(df.Start[i]) > 0: # and start and end are in the correct order
                                ax.barh(y = resource_position,
                                        width = mdates.date2num(df.End[i]) - mdates.date2num(df.Start[i]),
                                        left = mdates.date2num(df.Start[i]),
                                        height = 0.9,
                                        align='center',
                                        color = '#000000',
                                        alpha=0.3)
                            else:
                                print("Error: Dates incorrectly specified (ends before start)")
                        else:
                            print("Error: Start date has been specified, but end date has not")
        row_title_positions.append(resource_position)
        row_titles.append(resource)
        resource_position -= 1 # iterate in reverse
    
    # Add title for resources chunk
    row_title_positions.append(resource_position)
    row_titles.append(r'\textbf{Key resources allocations}')
    
    # Divider between resources and work packets
    ax.axhline(y = 0,
               linestyle = '-',
               color = '#000000',
               alpha = 1.0,
               linewidth = 0.75)

In [11]:
if True:
    ax.set_yticks(row_title_positions)
    ax.set_yticklabels(row_titles)

    ax.tick_params("y", left=False, labelleft=False)
    ax.tick_params("y", right=True, labelright=True)

    ax.grid(b = True,
            which = 'major',
            axis = 'x',
            linestyle = ':',
            color = 'tab:gray',
            alpha = 0.5,
            linewidth = 0.75)

    ax.grid(b = True,
            which = 'minor',
            axis = 'x',
            linestyle = ':',
            color = 'tab:gray',
            alpha = 0.2,
            linewidth = 0.75)

if True:
    # Plot bars
    ax.barh(y = bar_vertical_positions,
            width =mdates.date2num(work_package_end) - mdates.date2num(work_package_start),
            left = mdates.date2num(work_package_start),
            height = 0.9,
            align='center',
            color = bar_color)

if True:

    # Annotate each bar
    for i in range(len(work_package_start)):
        if type(work_package_annotations[i]) == str:
            ax.annotate(text = work_package_annotations[i],
                        xy = (mdates.date2num(work_package_midpoint[i]), bar_vertical_positions[i]),
                        xycoords = 'data',
                        horizontalalignment='center',
                        verticalalignment='center')

    for division_position in group_break_positions:
        print("Group divider placed at:", division_position)
        # Plot dashed lines between each group
        ax.axhline(y = division_position,
                   linestyle = ':',
                   color = 'tab:gray',
                   alpha = 0.5,
                   linewidth = 0.75)


    #plt.title('')
    #plt.xlabel('')
    #plt.ylabel('')

    plt.tight_layout()

if True:

    plt.xlim(left = np.min(work_package_start))
    plt.xlim(right = np.max(work_package_end))
    plt.ylim(bottom = np.min(row_title_positions))
    plt.ylim(top =np.max(row_title_positions) + 0.75)
    
    
if True:
    ax.invert_yaxis()  # labels read top-to-bottom

    # format xaxis with 3 month intervals
    ax.get_xaxis().set_major_locator(mdates.MonthLocator(interval=3))
    ax.get_xaxis().set_major_formatter(mdates.DateFormatter("%b \n %Y"))

    ax.get_xaxis().set_minor_locator(mdates.MonthLocator(interval=1))
    #ax.get_xaxis().set_minor_formatter(mdates.DateFormatter("%b %Y"))

    plt.setp(ax.get_xticklabels(), rotation=0, ha="center")
    ax.xaxis_date()



    ## recompute the ax.dataLim
    #ax.relim()
    ## update ax.viewLim using the new dataLim
    #ax.autoscale_view(tight=True)

#plt.show()
# Beware: output not show correctly below...look at saved files!

Group divider placed at: 0
Group divider placed at: 7
Group divider placed at: 16
Group divider placed at: 21
Group divider placed at: 24
Group divider placed at: 27


RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 432x288 with 1 Axes>

In [12]:
# Toggle save (takes time)
if True:
    fig.savefig('Gantt.png', dpi=300)
    fig.savefig('Gantt.pdf')
    plt.close('all')

RuntimeError: Failed to process string with tex because latex could not be found